In [38]:
!pip install psycopg2-binary
!pip install tabulate
import psycopg2 as pg
import pandas as pd

In [42]:
con = pg.connect("dbname=testing user=postgres password=password host=localhost port=5433")
cur = con.cursor()
cur.execute('''
select column_name
from information_schema.COLUMNS as c
where c.TABLE_NAME = 'input-avro' and c.COLUMN_NAME like 'meta_%'
''')
cols = [c for c, in cur.fetchall()]

fns = ['avg', 'min', 'max', 'stddev']
keys = [(fn, c) for fn in fns for c in cols]
sub = ', '.join(f'{fn}("{c}") as {fn}_{c}' for fn, c in keys)
q = f'select {sub} from "input-avro"'
cur.execute(q)
d = {c: [c] for c in cols}

for (fn, c), val in zip(keys, cur.fetchone()):
    d[c].append(val)
    

stats = pd.DataFrame(d.values(), columns = ['meta', *fns])
print(stats.to_markdown())

|    | meta                |      avg |   min |           max |   stddev |
|---:|:--------------------|---------:|------:|--------------:|---------:|
|  0 | meta_block          |  36791.8 |  4000 |   2.80018e+06 |  48476.9 |
|  1 | meta_cycle_time     |          |   nan | nan           |          |
|  2 | meta_dynamic        |  81886   | 12000 |   4.04025e+06 | 101087   |
|  3 | meta_info           | 101471   | 50999 |   1.92e+06    | 126015   |
|  4 | meta_message        |  39838.7 |  3935 |   2.66817e+06 |  52815.9 |
|  5 | meta_part_count     |  33850.7 |     0 |   6.67158e+06 |  45438.2 |
|  6 | meta_program        | 141192   | 10593 |   2.31614e+06 |  66434.7 |
|  7 | meta_program_header | 132309   |  8273 |   2.89131e+06 | 126097   |
|  8 | meta_program_name   |  37777.4 | 13188 |   2.64417e+06 |  31857.8 |
|  9 | meta_status         |  33599.2 |  3723 |   2.66417e+06 |  44925.1 |
| 10 | meta_tool           |  34783.6 |     0 |   2.51616e+06 |  46002.4 |
| 11 | meta_total        